In [1]:
# import libraries
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
import dash_daq as daq

import plotly.express as px

import plotly.io as pio
pio.renderers.default = 'notebook_connected'

In [2]:
COLOR_SET = px.colors.qualitative.Plotly

COLOR_BAR = ['indianred']

COLOR_SET

['#636EFA',
 '#EF553B',
 '#00CC96',
 '#AB63FA',
 '#FFA15A',
 '#19D3F3',
 '#FF6692',
 '#B6E880',
 '#FF97FF',
 '#FECB52']

In [3]:
df = pd.read_csv('IMDb_movies.csv')

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3441: DtypeWarning:

Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.



In [4]:
df.shape

(85855, 22)

In [5]:
df.head(5)

,imdb_title_id,title,original_title,year,date_published,genre,duration,country,language,director,...,actors,description,avg_vote,votes,budget,usa_gross_income,worlwide_gross_income,metascore,reviews_from_users,reviews_from_critics
0,tt0000009,Miss Jerry,Miss Jerry,1894,1894-10-09,Romance,45,USA,None,Alexander Black,...,"Blanche Bayliss, William Courtenay, Chauncey D...",The adventures of a female reporter in the 1890s.,5.9,154,NaN,NaN,NaN,NaN,1.0,2.0
1,tt0000574,The Story of the Kelly Gang,The Story of the Kelly Gang,1906,1906-12-26,"Biography, Crime, Drama",70,Australia,None,Charles Tait,...,"Elizabeth Tait, John Tait, Norman Campbell, Be...",True story of notorious Australian outlaw Ned ...,6.1,589,$ 2250,NaN,NaN,NaN,7.0,7.0
2,tt0001892,Den sorte drøm,Den sorte drøm,1911,1911-08-19,Drama,53,"Germany, Denmark",NaN,Urban Gad,...,"Asta Nielsen, Valdemar Psilander, Gunnar Helse...",Two men of high rank are both wooing the beaut...,5.8,188,NaN,NaN,NaN,NaN,5.0,2.0
3,tt0002101,Cleopatra,Cleopatra,1912,1912-11-13,"Drama, History",100,USA,English,Charles L. Gaskill,...,"Helen Gardner, Pearl Sindelar, Miss Fielding, ...",The fabled queen of Egypt's affair with Roman ...,5.2,446,$ 45000,NaN,NaN,NaN,25.0,3.0
4,tt0002130,L'Inferno,L'Inferno,1911,1911-03-06,"Adventure, Drama, Fantasy",68,Italy,Italian,"Francesco Bertolini, Adolfo Padovan",...,"Salvatore Papa, Arturo Pirovano, Giuseppe de L...",Loosely adapted from Dante's Divine Comedy and...,7.0,2237,NaN,NaN,NaN,NaN,31.0,14.0


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 85855 entries, 0 to 85854
Data columns (total 22 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   imdb_title_id          85855 non-null  object 
 1   title                  85855 non-null  object 
 2   original_title         85855 non-null  object 
 3   year                   85855 non-null  object 
 4   date_published         85855 non-null  object 
 5   genre                  85855 non-null  object 
 6   duration               85855 non-null  int64  
 7   country                85791 non-null  object 
 8   language               85022 non-null  object 
 9   director               85768 non-null  object 
 10  writer                 84283 non-null  object 
 11  production_company     81400 non-null  object 
 12  actors                 85786 non-null  object 
 13  description            83740 non-null  object 
 14  avg_vote               85855 non-null  float64
 15  vo

In [7]:
df.isna().sum()

imdb_title_id                0
title                        0
original_title               0
year                         0
date_published               0
genre                        0
duration                     0
country                     64
language                   833
director                    87
writer                    1572
production_company        4455
actors                      69
description               2115
avg_vote                     0
votes                        0
budget                   62145
usa_gross_income         70529
worlwide_gross_income    54839
metascore                72550
reviews_from_users        7597
reviews_from_critics     11797
dtype: int64

In [8]:
df.date_published = pd.to_datetime(df.date_published, errors='coerce')
df['Year'] = df.date_published.dt.year
df.dropna(subset=['Year','country'], inplace=True)
df.sort_values('Year', inplace=True)

In [9]:
df.budget = pd.to_numeric(df.budget.str.replace("[^\d]",'').str.strip())
df.usa_gross_income = pd.to_numeric(df.usa_gross_income.str.replace("[^\d]",'').str.strip())
df.worlwide_gross_income = pd.to_numeric(df.worlwide_gross_income.str.replace("[^\d]",'').str.strip())

<ipython-input-9-75c71a0c489c>:1: FutureWarning:

The default value of regex will change from True to False in a future version.

<ipython-input-9-75c71a0c489c>:2: FutureWarning:

The default value of regex will change from True to False in a future version.

<ipython-input-9-75c71a0c489c>:3: FutureWarning:

The default value of regex will change from True to False in a future version.



In [10]:
df['Revenue'] = df.worlwide_gross_income - df.budget
df['Genre'] = pd.DataFrame(df.genre.str.split(',').tolist(), index= df.index)[0]
df['Country'] = pd.DataFrame(df.country.str.split(',').tolist(), index= df.index)[0]


In [11]:
def myFunc(x):
    try:
        x = x.split(',')
        if isinstance(x, list):
            return x[0]
    except:
        return np.nan

df['Language'] = df.language.replace('None', np.nan).apply(myFunc)

In [12]:
df.rename(columns={"avg_vote": "Average Vote",
                    'votes':'Vote Count',
                    'budget':'Budget',
                    'duration':'Duration',
                    'title':'Title'},
                    inplace=True)

In [13]:
df = df[df['Year'] < 2021]

In [14]:
year_filter = df['Year'] == 2015
year_filter

0        False
1        False
2        False
4        False
39150    False
         ...  
65308    False
83456    False
83458    False
54127    False
80436    False
Name: Year, Length: 85783, dtype: bool

In [15]:
df.describe()

,Duration,Average Vote,Vote Count,Budget,usa_gross_income,worlwide_gross_income,metascore,reviews_from_users,reviews_from_critics,Year,Revenue
count,85783.000000,85783.000000,8.578300e+04,2.370400e+04,1.532100e+04,3.100000e+04,13298.000000,78193.000000,74004.000000,85783.000000,1.276100e+04
mean,100.353054,5.899472,9.500935e+03,1.057386e+08,1.949934e+07,2.309439e+07,55.898406,46.066732,27.493284,1993.877237,-3.109093e+07
std,22.553487,1.234040,5.359616e+04,2.771097e+09,4.825059e+07,9.019339e+07,17.778943,178.582230,58.356924,24.168837,1.368940e+09
min,41.000000,1.000000,9.900000e+01,0.000000e+00,3.000000e+01,1.000000e+00,1.000000,1.000000,1.000000,1894.000000,-1.199986e+11
25%,88.000000,5.200000,2.050000e+02,7.991400e+05,9.329200e+04,1.152982e+05,43.000000,4.000000,3.000000,1979.000000,-5.724443e+06
50%,96.000000,6.100000,4.850000e+02,3.000000e+06,1.310470e+06,1.115724e+06,57.000000,9.000000,8.000000,2003.000000,-6.783130e+05
75%,108.000000,6.800000,1.769000e+03,1.500000e+07,1.712903e+07,8.444134e+06,69.000000,27.000000,24.000000,2013.000000,1.397200e+07
max,808.000000,9.900000,2.278845e+06,3.500000e+11,9.366622e+08,2.797801e+09,100.000000,10472.000000,999.000000,2020.000000,2.553439e+09


In [16]:
gy = df.groupby('Year').mean().reset_index()
gy

,Year,Duration,Average Vote,Vote Count,Budget,usa_gross_income,worlwide_gross_income,metascore,reviews_from_users,reviews_from_critics,Revenue
0,1894.0,45.000000,5.900000,154.000000,NaN,NaN,NaN,NaN,1.000000,2.000000,NaN
1,1906.0,70.000000,6.100000,589.000000,2.250000e+03,NaN,NaN,NaN,7.000000,7.000000,NaN
2,1911.0,70.200000,6.140000,566.600000,NaN,NaN,NaN,NaN,10.500000,8.000000,NaN
3,1912.0,80.000000,6.050000,293.500000,1.583333e+05,NaN,NaN,NaN,10.750000,3.250000,NaN
4,1913.0,83.461538,6.438462,720.307692,2.535000e+04,NaN,NaN,NaN,9.615385,12.000000,NaN
...,...,...,...,...,...,...,...,...,...,...,...
107,2016.0,101.529094,5.640509,9228.160572,2.227268e+08,2.122069e+07,2.196011e+07,54.823432,40.406904,35.361856,-1.041554e+08
108,2017.0,102.388358,5.697970,8337.724478,1.796182e+08,2.136117e+07,2.312662e+07,57.500785,50.068826,33.828259,-5.657132e+07
109,2018.0,103.242181,5.688353,6893.179923,1.971560e+08,2.121503e+07,2.153497e+07,58.331825,68.101745,30.239234,-1.978203e+08
110,2019.0,104.666452,5.782710,6634.757097,5.040580e+08,2.715121e+07,2.361137e+07,56.383361,93.434954,29.640000,-6.846059e+07


In [17]:
px.line(gy, x='Year', y=['Average Vote'])

In [18]:
px.line(gy, x='Year', y=[ 'Vote Count'])

In [19]:
# import some stuff
from plotly.subplots import make_subplots

subfig = make_subplots(specs=[[{"secondary_y": True}]])

# create two independent figures with px.line each containing data from multiple columns
fig = px.line(gy, x='Year', y=['Average Vote'], render_mode="webgl",)
fig2 = px.line(gy, x='Year', y=[ 'Vote Count'], render_mode="webgl",)

fig2.update_traces(yaxis="y2")

subfig.add_traces(fig.data + fig2.data)
subfig.layout.xaxis.title="Year"
subfig.layout.yaxis.title="Vote"
# subfig.layout.yaxis2.type="log"
subfig.layout.yaxis2.title="Vote Counts"
# recoloring is necessary otherwise lines from fig und fig2 would share each color
# e.g. Linear-, Log- = blue; Linear+, Log+ = red... we don't want this
subfig.for_each_trace(lambda t: t.update(line=dict(color=t.marker.color)))
subfig.show()

In [20]:
px.line(df.groupby('Year').mean().reset_index(), x='Year', y=['Duration'])

In [21]:
px.line(gy, x='Year', y=['Budget','Revenue'], )

In [22]:
df_temp = df[df.language == 'English']
grouped_df = df_temp.groupby(['Year']).mean().reset_index()
grouped_df

px.area(grouped_df, x='Year', y=['Budget','Revenue'])

In [23]:
def plot_scatter_graph(dataframe, template='ggplot2'):
    return px.scatter(dataframe, x='Budget', y ='Average Vote',
                    size='Vote Count', color = 'Genre',
                    log_x =True, size_max=60,
                    hover_name='Title',hover_data=['Duration','director','language','country'],
                    color_discrete_sequence=COLOR_SET,
                    template=template).update_layout( title={
                            'text': "Average Vote - Budget",
                            'y':0.97,
                            'x':0.5,
                            'xanchor': 'center',
                            'yanchor': 'top'
                    } )

scatter_graph = plot_scatter_graph(df[year_filter])
scatter_graph

In [24]:
def plot_hist_duration(dataframe, ):
    return px.histogram(dataframe, x='Duration', color_discrete_sequence=COLOR_BAR).update_layout( title={
                            'text': "Duration Histogram",
                            'y':0.97,
                            'x':0.5,
                            'xanchor': 'center',
                            'yanchor': 'top'
                    } )

hist_duration = plot_hist_duration(df[year_filter])
hist_duration

In [25]:
def plot_treemap(dataframe,  height=600):
    return px.treemap(dataframe, values='Average Vote', path=['Genre', 'Country'], hover_name='Country', height=height, color_discrete_sequence=COLOR_SET)

treemap = plot_treemap(df[year_filter])
treemap

In [26]:
def plot_sunbrust(dataframe,  height=600):
    return px.sunburst(dataframe, values='Average Vote', path=['Genre', 'Country'], hover_name='Country', color_discrete_sequence=COLOR_SET).update_layout( title={
                            'text': "Countries per Genres ",
                            'y':0.978,
                            'x':0.5,
                            'xanchor': 'center',
                            'yanchor': 'top'
                    } )

sunbrust = plot_sunbrust(df[year_filter])
sunbrust

In [27]:
def plot_bar_title_length(dataframe):
    counts, bins = np.histogram(dataframe.Title.str.replace(' ', ',').str.split(',').apply(lambda x:len(x)), bins=range(1, 14, 1))
    bins = 0.5 * (bins[:-1] + bins[1:])
    return px.bar(x=bins, y=counts, labels={'x':'Title Length (Words)', 'y':'count'}, color_discrete_sequence=COLOR_BAR)

bar_title_length = plot_bar_title_length(df[year_filter])
bar_title_length

In [28]:
def plot_bar_language(dataframe,  height=600):
    lang_df = dataframe.groupby('Language').size().sort_values(ascending=False).reset_index(name='counts')
    return px.bar(lang_df[:10], x='Language', y='counts',  barmode='group', log_y=True, color_discrete_sequence=COLOR_BAR)

bar_language = plot_bar_language(df[year_filter])
bar_language

In [29]:
def plot_bar_genre(dataframe, ):
    genre_df = dataframe.groupby('Genre').size().sort_values(ascending=False).reset_index(name='counts')
    return px.bar(genre_df[genre_df.counts >5], x='Genre', y='counts', color='Genre', color_discrete_sequence=COLOR_SET).update_layout(showlegend=False)
    # return px.bar(genre_df[genre_df.counts >5], x='Genre', y='counts',  barmode='group', color_discrete_sequence=COLOR_BAR)

bar_genre = plot_bar_genre(df[year_filter])
bar_genre

In [30]:
def plot_bar_country(dataframe, ):
    country_df = dataframe.groupby(by=['country']).size().reset_index(name='counts').sort_values(by=['counts'], ascending=False)[:10]
    return px.bar(data_frame=country_df, x='country', y='counts', barmode='group', log_y=True, color_discrete_sequence=COLOR_BAR)

bar_country = plot_bar_country(df[year_filter])
bar_country

In [31]:
yg = df.groupby(['Year']).mean().reset_index()
yg

,Year,Duration,Average Vote,Vote Count,Budget,usa_gross_income,worlwide_gross_income,metascore,reviews_from_users,reviews_from_critics,Revenue
0,1894.0,45.000000,5.900000,154.000000,NaN,NaN,NaN,NaN,1.000000,2.000000,NaN
1,1906.0,70.000000,6.100000,589.000000,2.250000e+03,NaN,NaN,NaN,7.000000,7.000000,NaN
2,1911.0,70.200000,6.140000,566.600000,NaN,NaN,NaN,NaN,10.500000,8.000000,NaN
3,1912.0,80.000000,6.050000,293.500000,1.583333e+05,NaN,NaN,NaN,10.750000,3.250000,NaN
4,1913.0,83.461538,6.438462,720.307692,2.535000e+04,NaN,NaN,NaN,9.615385,12.000000,NaN
...,...,...,...,...,...,...,...,...,...,...,...
107,2016.0,101.529094,5.640509,9228.160572,2.227268e+08,2.122069e+07,2.196011e+07,54.823432,40.406904,35.361856,-1.041554e+08
108,2017.0,102.388358,5.697970,8337.724478,1.796182e+08,2.136117e+07,2.312662e+07,57.500785,50.068826,33.828259,-5.657132e+07
109,2018.0,103.242181,5.688353,6893.179923,1.971560e+08,2.121503e+07,2.153497e+07,58.331825,68.101745,30.239234,-1.978203e+08
110,2019.0,104.666452,5.782710,6634.757097,5.040580e+08,2.715121e+07,2.361137e+07,56.383361,93.434954,29.640000,-6.846059e+07


In [32]:
ygg =df.groupby(['Year','Genre']).mean().reset_index()
ygg

,Year,Genre,Duration,Average Vote,Vote Count,Budget,usa_gross_income,worlwide_gross_income,metascore,reviews_from_users,reviews_from_critics,Revenue
0,1894.0,Romance,45.000000,5.900000,154.000000,NaN,NaN,NaN,NaN,1.000000,2.000000,NaN
1,1906.0,Biography,70.000000,6.100000,589.000000,2.250000e+03,NaN,NaN,NaN,7.000000,7.000000,NaN
2,1911.0,Adventure,68.000000,7.000000,2237.000000,NaN,NaN,NaN,NaN,31.000000,14.000000,NaN
3,1911.0,Drama,66.500000,6.000000,175.000000,NaN,NaN,NaN,NaN,5.000000,2.000000,NaN
4,1911.0,Fantasy,50.000000,5.700000,102.000000,NaN,NaN,NaN,NaN,1.000000,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
1692,2020.0,Sci-Fi,91.588235,4.317647,669.411765,2.485714e+05,139593.0,1.147970e+05,70.666667,30.294118,11.400000,NaN
1693,2020.0,Sport,104.000000,6.000000,700.000000,NaN,NaN,NaN,NaN,24.000000,6.000000,NaN
1694,2020.0,Thriller,94.693878,5.124490,692.122449,6.864286e+06,1941.0,9.483702e+05,49.750000,20.720930,8.023256,-1477236.0
1695,2020.0,War,116.333333,5.066667,436.666667,8.000000e+07,111552.0,2.772073e+08,NaN,17.666667,3.666667,197207347.0


In [33]:
ygg[ygg.Year == 2000]

,Year,Genre,Duration,Average Vote,Vote Count,Budget,usa_gross_income,worlwide_gross_income,metascore,reviews_from_users,reviews_from_critics,Revenue
1310,2000.0,Action,109.900000,5.202941,33441.052941,1.431543e+08,4.071484e+07,7.670616e+07,47.564103,131.878788,34.423077,6.625842e+07
1311,2000.0,Adventure,99.514286,5.737143,29017.314286,3.802941e+07,3.413069e+07,5.102989e+07,45.764706,147.500000,41.121212,2.043449e+07
1312,2000.0,Animation,81.361111,6.422222,29373.361111,2.295000e+08,5.807794e+07,1.032724e+08,53.307692,121.393939,47.580645,-1.185975e+08
1313,2000.0,Biography,116.676471,6.782353,27768.705882,6.196615e+07,1.717488e+07,2.694187e+07,65.937500,113.470588,44.625000,1.064019e+07
1314,2000.0,Comedy,99.769953,5.789202,8365.446009,8.266944e+07,1.423060e+07,2.111869e+07,49.115108,51.066158,24.980226,8.731179e+06
1315,2000.0,Crime,101.516129,5.829032,22884.344086,1.569216e+07,1.651886e+07,3.174564e+07,49.035714,91.430233,31.378049,3.282987e+07
1316,2000.0,Drama,105.908654,6.175962,9582.817308,1.674624e+07,7.897972e+06,1.628937e+07,57.672269,52.135338,23.285285,6.699992e+06
1317,2000.0,Family,90.714286,5.423810,291.047619,7.500000e+06,2.273740e+05,4.400245e+05,NaN,5.411765,3.250000,NaN
1318,2000.0,Fantasy,96.400000,5.420000,64803.800000,1.000000e+08,1.010715e+08,2.060715e+08,65.000000,210.200000,81.666667,1.060715e+08
1319,2000.0,Horror,90.900000,4.578000,20017.340000,1.245854e+07,3.736787e+07,6.022731e+07,42.416667,180.408163,51.468085,4.688601e+07


In [34]:
px.line(ygg[(ygg.Genre.isin(['Action', 'Comedy', 'Drama' ]))&(ygg.Year < 2020)&(ygg.Year > 1930)], x='Year', y='Average Vote', color='Genre')

In [35]:
cg = df[year_filter].groupby('Country').mean().reset_index()
cg

,Country,Duration,Average Vote,Vote Count,Budget,usa_gross_income,worlwide_gross_income,metascore,reviews_from_users,reviews_from_critics,Year,Revenue
0,Albania,112.000000,5.550000,197.000000,85000.0,NaN,NaN,NaN,2.5,9.000000,2015.0,NaN
1,Algeria,90.000000,3.600000,178.000000,2300000.0,NaN,NaN,NaN,4.0,9.000000,2015.0,NaN
2,Argentina,91.800000,5.760000,464.960000,4150000.0,NaN,6.410272e+05,46.0,2.8,8.875000,2015.0,-11939642.00
3,Australia,96.085106,5.576596,31077.638298,16970919.0,1.991646e+07,2.081780e+07,61.5,100.0,62.613636,2015.0,12857508.75
4,Austria,102.833333,6.600000,7480.333333,NaN,1.178196e+06,1.018330e+06,81.0,44.0,57.666667,2015.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
80,United Arab Emirates,98.000000,6.950000,5247.500000,NaN,2.830240e+05,4.292160e+05,80.0,25.0,63.000000,2015.0,NaN
81,Uruguay,89.000000,6.450000,247.500000,NaN,NaN,5.480150e+04,NaN,NaN,7.000000,2015.0,NaN
82,Venezuela,97.000000,6.500000,204.000000,NaN,NaN,NaN,NaN,2.0,5.000000,2015.0,NaN
83,Vietnam,109.666667,7.233333,527.666667,1100000.0,7.177500e+04,3.552048e+06,NaN,4.5,3.666667,2015.0,2193667.00


In [36]:
import pycountry

countries = {}
for country in pycountry.countries:
    countries[country.name] = country.alpha_3

In [37]:
un_iso = {'Bolivia': 'BOL',
        'Brunei':	'BRN' ,
        'Czech Republic' : 'CZE'  ,
        'Czechoslovakia':'CZE'  ,
        'East Germany':	'DEU'  ,
        'Federal Republic of Yugoslavia': 'YUG' ,
        'Iran' : 'IRN',
        'Isle Of Man': 'IMN' ,
        'Korea': 'KOR',
        'Kosovo': 'XXK',
        'Laos': 'LAO',
        'Moldova': 'MDA',
        'Netherlands Antilles': 'ANT',
        'North Korea':'PRK' ,
        'North Vietnam': 'VDR'  ,
        'Palestine': 'PSE' ,
        'Republic of North Macedonia' : 'MKD' ,
        'Russia': 'RUS',
        'Serbia and Montenegro': 'SCG' ,
        'South Korea': 'KOR' ,
        'Soviet Union': 'SUN'  ,
        'Syria': 'SYR',
        'Taiwan': 'zho',
        'Tanzania': 'TZA' ,
        'The Democratic Republic Of Congo': 'COD',
        'UK': 'GBR',
        'USA': 'USA',
        'Venezuela': 'VEN' ,
        'Vietnam': 'VNM',
        'West Germany' :'DEU'  ,
        'Yugoslavia': 'YUG' }

countries.update(un_iso)

codes = [countries.get(country, 'Unknown code') for country in cg.Country]
print(codes)

['ALB', 'DZA', 'ARG', 'AUS', 'AUT', 'AZE', 'BHS', 'BGD', 'BEL', 'BIH', 'BRA', 'BGR', 'CAN', 'CHL', 'CHN', 'COL', 'CRI', 'HRV', 'CUB', 'CZE', 'DNK', 'DOM', 'EGY', 'EST', 'ETH', 'FIN', 'FRA', 'GEO', 'DEU', 'GRC', 'GTM', 'HKG', 'HUN', 'ISL', 'IND', 'IDN', 'IRN', 'IRQ', 'IRL', 'ISR', 'ITA', 'JPN', 'LVA', 'LTU', 'LUX', 'MYS', 'MRT', 'MEX', 'MNE', 'NPL', 'NLD', 'NZL', 'NGA', 'NOR', 'PAK', 'PER', 'PHL', 'POL', 'PRT', 'PRI', 'QAT', 'MKD', 'ROU', 'RUS', 'SRB', 'SGP', 'SVK', 'SVN', 'ZAF', 'KOR', 'ESP', 'LKA', 'SWE', 'CHE', 'zho', 'THA', 'TUR', 'GBR', 'USA', 'UKR', 'ARE', 'URY', 'VEN', 'VNM', 'YEM']


In [38]:
cg['iso_code'] = codes

# check
cg[cg.iso_code == 'Unknown code'][['Country','iso_code']]

,Country,iso_code


In [39]:
df[['Country', 'Average Vote']]

,Country,Average Vote
0,USA,5.9
1,Australia,6.1
2,Germany,5.8
4,Italy,7.0
39150,Serbia,6.2
...,...,...
65308,USA,6.0
83456,USA,4.7
83458,USA,3.0
54127,USA,5.7


In [40]:
def plot_choropleth_map(dataframe, title=None, height=600):
    # check-me MAX
    cg = dataframe.groupby('Country').max().reset_index()
    cg['iso_code'] = [countries.get(country, 'Unknown code') for country in cg.Country]

    return px.choropleth(cg, color ='Average Vote', locations='iso_code', hover_name='Country').update_layout(
                            title={
                            'text': "Maximum Vote for each Country",
                            'y':0.97,
                            'x':0.5,
                            'xanchor': 'center',
                            'yanchor': 'top'}, 
                            geo=dict(bgcolor= 'rgba(0,0,0,0)'),
                            coloraxis = {'colorbar':{'title': 'Vote'}})

# df2 = df[['Country', 'Average Vote']]
choropleth_map = plot_choropleth_map(df[year_filter])
choropleth_map

<ipython-input-40-a53fd05f5f7b>:3: FutureWarning:

Dropping invalid columns in DataFrameGroupBy.max is deprecated. In a future version, a TypeError will be raised. Before calling .max, select only columns which should be valid for the function.



===============================================================================================================
# DASH 
===============================================================================================================

In [41]:
import dash
from jupyter_dash import JupyterDash
from dash import html
from dash import dcc
import plotly.express as px
from dash.dependencies import Input, Output, State

import dash_bootstrap_components as dbc

In [42]:
# Draw figure Function
def drawFigure(figure=None, id=None, config={'displayModeBar': False, 'autosizable':True, 'responsive':True}):
    return dcc.Graph(
                    id=id,
                    figure=figure.update_layout(
                        template     = 'plotly_dark',
                        plot_bgcolor = 'rgba(0, 0, 0, 0)',
                        paper_bgcolor= 'rgba(0, 0, 0, 0)',
                        autosize=True,
                        margin=dict(l=20, r=20, t=40, b=20),
                    ),
                    config=config,
                    style={'background-color':'#32383E', 'height':'100%'}
                )

In [43]:
# Text field Function
def drawText(text='My Text', style={'textAlign': 'center'}):
    return html.Div([
        html.H1(text, style={
                'flex':'0', 
                'margin-bottom':'36px',
                'font-size': '45px',
                'color': 'white'
            }),
    ], style={'padding-top':'18px'})

In [44]:
# Build App
app = JupyterDash(external_stylesheets=[dbc.themes.SLATE])

In [45]:
dbc.themes

<module 'dash_bootstrap_components.themes' from 'C:\\ProgramData\\Anaconda3\\lib\\site-packages\\dash_bootstrap_components\\themes.py'>

In [46]:
# Custom Style
NAVBAR_STYLE = {
    'height': "200px",
}

ROW_STYLE = {
    'height': "450px",
}

TITLE_STYLE={
    'height': '130px', 
    'flex':'0',
    'margin':'0 auto',
    'textAlign': 'center',
    'margin-bottom':'10px'
}

In [47]:
import base64

image_filename = 'logo4.png' 
encoded_image  = base64.b64encode(open(image_filename, 'rb').read()).decode("utf-8")

In [48]:
app.layout = html.Div([
    dbc.Card(
        dbc.CardBody([
            # ------------------------------------------ Bans Section  ------------------------------------------ #
            dbc.Row([
                dbc.Col([
                    html.Img(src='data:image/png;base64,{}'.format(encoded_image), height=120, style={'margin-bottom':'20px'}),
                    html.H1("Internet Movie Database", style={'font-size': '20px','color': 'white'}),
                    daq.Slider(
                        id='slider_year',
                        min=df['Year'].min(),
                        max=df['Year'].max(),
                        step=None,
                        value=2015,
                        color='#bd9321',
                        marks = { str(i):str(i) for i in df['Year'].unique() },
                        handleLabel={"label": "Year", "showCurrentValue": True, 'style':{'height':'33px', 'top':'20px', 'font-size':'10px'}},
                        labelPosition='top'
                    ),
                    html.Div(id='slider-output-container')
                ], width=2, style={'text-align': 'center'}),
                #===========##===========##===========##===========##===========#
                dbc.Col([
                    dbc.Row([
                        dbc.Col([
                            html.H6("World Wide Gross", style={'font-size': '20px','color': 'white', 'font-family':'Sans-serif'}),
                            html.H1("${:,.0f}M".format(df[year_filter]['worlwide_gross_income'].sum()/1000000), id='gross', style={'font-size': '30px','color': 'white'}),
                        ], width=4, style={"height": "180px", 'font-family': 'Cursive', "background-color": "#37597d", 'text-align':'center', 'justify':'center', 'align':'center', 'padding-top': '40px'}),

                        dbc.Col([
                            html.H6("Total Budget", style={'font-size': '20px','color': 'white', 'font-family':'Sans-serif'}),
                            html.H1("${:,.0f}M".format(df[year_filter]['Budget'].sum()/1000000), id='budget', style={'font-size': '30px','color': 'white'}),
                        ], width=4, style={"height": "180px", 'font-family': 'Cursive', "background-color": "#e37900", 'text-align':'center', 'justify':'center', 'align':'center', 'padding-top': '40px'}),
                        
                        dbc.Col([
                            html.H6("Year", style={'font-size': '20px','color': 'white', 'font-family':'Sans-serif'}),
                            html.H1("2015", id='year', style={'font-size': '40px','color': 'white'}),
                        ], width=4, style={"height": "180px", 'font-family': 'Cursive', "background-color": "#e24b3c", 'text-align':'center', 'justify':'center', 'align':'center', 'padding-top': '40px'}),
                    ]),
                ], width=4), 
                #===========##===========##===========##===========##===========#
                dbc.Col([
                    # drawFigure(id='subfig' ,figure=subfig)
                    dcc.Graph(
                    id='subfig',
                    figure=subfig.update_layout(
                        template     = 'plotly_dark',
                        plot_bgcolor = 'rgba(0, 0, 0, 0)',
                        paper_bgcolor= 'rgba(0, 0, 0, 0)',
                        autosize=True,
                        margin=dict(l=20, r=20, t=20, b=20),
                    ),
                    config={'displayModeBar': False, 'autosizable':True, 'responsive':True},
                    style={'background-color':'#32383E', 'height':'100%'}
                ) ], width=6, style={"height": "180px"}),
            ], align='center', style= NAVBAR_STYLE),

            html.Br(),
            # ------------------------------------------ Graph Section 1  ------------------------------------------ #
            dbc.Row([
                dbc.Col([
                    # dcc.Graph(id='scatter_graph')
                    drawFigure(id='scatter_graph' ,figure=scatter_graph)
                ], width=9,  style={"height": "100%"}),
                #===========##===========##===========##===========##===========#
                dbc.Col([
                    drawFigure(id='hist_duration' ,figure=hist_duration)
                ], width=3,  style={"height": "100%"}),
            ], align='center', style=ROW_STYLE),

            html.Br(),
            # ------------------------------------------ Graph Section 2  ------------------------------------------ #
            dbc.Row([
                dbc.Col([
                    dbc.Tabs(
                        [
                            dbc.Tab( label="Genre", children=[drawFigure(id='bar_genre', figure=bar_genre)]),
                            dbc.Tab( label="Country", children=[drawFigure(id='bar_country', figure=bar_country)]),
                            dbc.Tab( label="Language", children=[drawFigure(id='bar_language', figure=bar_language)]),
                            dbc.Tab( label="Title Length", children=[drawFigure(id='bar_title_length', figure=bar_title_length)]),
                        ], style={'height':'25px', 'position':'absolute', 'right':'50px','z-index':'5'}
                    )
                ], width=3, style={"height": "100%", 'position':'relative'}),
                #===========##===========##===========##===========##===========#
                dbc.Col([
                    drawFigure(id='sunbrust' ,figure=sunbrust)
                ], width=3,  style={"height": "100%"}),
                #===========##===========##===========##===========##===========#
                dbc.Col([
                    drawFigure(id='choropleth_map', figure=choropleth_map)
                ], width=6,  style={"height": "100%"}),
            ], align='center', style=ROW_STYLE),
            
            html.Br(),
            # ------------------------------------------ About Section   ------------------------------------------ #
            dbc.Row([
                dbc.Col([

                ], width=3,  style={"height": "100%", }),
                dbc.Col([
                    html.H3("Developers", style={'font-size': '20px','color': 'white', 'font-family':'Sans-serif'}),
                ], width=2,  style={"height": "100%", }),
                dbc.Col([
                    html.A(['Ahmed N. Awaad'], href='https://www.linkedin.com/in/ahmed-n-awaad/',target='_blank', style={'font-size': '20px','color': 'white', 'font-family':'Sans-serif'}),
                ], width=2,  style={"height": "100%", }),
                dbc.Col([
                    html.A(['Mohamed ELMesawy'], href='https://www.linkedin.com/in/mohamed-elmesawy/',target='_blank', style={'font-size': '20px','color': 'white', 'font-family':'Sans-serif'}),
                ], width=2,  style={"height": "100%", }),
                #===========##===========##===========##===========##===========#
            ], align='center',  style={"height": "30px"}),
        ]), color = 'dark'
    )
])

In [49]:
@app.callback(
    Output(component_id='scatter_graph', component_property='figure'),
    Output(component_id='choropleth_map', component_property='figure'),
    Output(component_id='sunbrust', component_property='figure'),
    Output(component_id='hist_duration', component_property='figure'),
    Output(component_id='bar_genre', component_property='figure'),
    Output(component_id='bar_country', component_property='figure'),
    Output(component_id='bar_language', component_property='figure'),
    Output(component_id='bar_title_length', component_property='figure'),

    Output(component_id='gross', component_property='children'),
    Output(component_id='budget', component_property='children'),
    Output(component_id='year', component_property='children'),
    
    Input(component_id='slider_year', component_property='value')
)
def update_graph_by_year_slider(input_slider):
    # generate filtered dataframe
    filtered_dataframe = df[df['Year'] == float(input_slider)]

    # Create plots
    scatter_graph    = plot_scatter_graph(filtered_dataframe)
    choropleth_map   = plot_choropleth_map(filtered_dataframe)
    sunbrust         = plot_sunbrust(filtered_dataframe)
    hist_duration    = plot_hist_duration(filtered_dataframe)
    bar_genre        = plot_bar_genre(filtered_dataframe)
    bar_country      = plot_bar_country(filtered_dataframe)
    bar_language     = plot_bar_language(filtered_dataframe)
    bar_title_length = plot_bar_title_length(filtered_dataframe)

    # Iterate over new plots and update layout
    graphs_list = [scatter_graph, choropleth_map, sunbrust, hist_duration, bar_genre, bar_country, bar_language, bar_title_length]
    for graph in graphs_list:
        graph.update_layout(
            template     = 'plotly_dark',
            plot_bgcolor = 'rgba(0, 0, 0, 0)',
            paper_bgcolor= 'rgba(0, 0, 0, 0)',
            autosize=True,
            margin=dict(l=20, r=20, t=40, b=20),
            font_family="Sans-serif",
            font_color="white",
            title_font_family="Sans-serif",
            title_font_color="white",
            legend_title_font_color="white"
        )
        
    # Update Bans
    budget = filtered_dataframe['Budget'].sum()/1000000
    gross = filtered_dataframe['worlwide_gross_income'].sum()/1000000
    
    graphs_list += ["${:,.0f}M".format(gross),  "${:,.0f}M".format(budget), str(input_slider)]
        
    return  tuple(graphs_list)

In [50]:
if __name__ == "__main__":
    app.run_server(port=9090)

Dash app running on http://127.0.0.1:9090/


<ipython-input-40-a53fd05f5f7b>:3: FutureWarning:

Dropping invalid columns in DataFrameGroupBy.max is deprecated. In a future version, a TypeError will be raised. Before calling .max, select only columns which should be valid for the function.

